# Imports

In [1]:
import yfinance as yf
import pandas as pd
import requests_cache
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter

In [3]:
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(5, Duration.SECOND*5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

# Finance Data

In [4]:
def get_stock_info(ticker: str, length: int):
    assert length in [1, 7, 30, 90, 365]
    stock = yf.Ticker(ticker, session=session)
    end_date = pd.Timestamp.today().strftime('%Y-%m-%d')
    start_date = (pd.Timestamp.today() - pd.Timedelta(days=length)).strftime('%Y-%m-%d')
    data  = {
        # Descriptive
        "ticker" : ticker,
        "name": stock.info["shortName"],
        "webiste": stock.info["website"],
        "sector": stock.info["sector"],
        "description": stock.info["longBusinessSummary"],
        "marketCap" : stock.info["marketCap"],
        "currency": stock.info["currency"],
        "info": stock.info,
        "price_history" : stock.history(start=start_date, end=end_date),
        "income_statement": stock.income_stmt,
        "balance_sheet": stock.balance_sheet,
        "cash_flow": stock.cashflow,
        "recommendations": stock.recommendations,
        "news": stock.news,
            }
    return data

print(get_stock_info('MSFT', 365))

{'ticker': 'MSFT', 'name': 'Microsoft Corporation', 'webiste': 'https://www.microsoft.com', 'sector': 'Technology', 'description': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power apps, and power automate; and on-premises ERP and CRM applications. The Intelligent Cloud segment offers server products and cloud services, such as azure and other cloud services; SQL and windows server, visual studio, system center, and related client access licenses, as well as nuance and G

# Finance LLM   

In [ ]:
# https://ollama.com/jamesbrink/finance-chat
